# **Spotify API**

In [1]:
pip install nose

In [2]:
import requests as rq
import pandas as pd

**Retrieving acces token & initializing**

In [3]:
client_id = 'id'
client_secret = 'secret'

In [4]:
auth_url = 'https://accounts.spotify.com/api/token'

params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}

auth_response = rq.post(auth_url, params).json()

access_token = auth_response['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

**Retrieving Radiohead - Creep (as test)**

In [5]:
base_url = 'https://api.spotify.com/v1/'
endpoint = 'audio-features'
url = base_url + endpoint

In [6]:
track_id = '6b2oQwSGFkzsMtQruIWm2p'
params = {'ids': [track_id]}
creep = rq.get(url, params = params, headers = headers).json()

In [7]:
from nose.tools import assert_equal
assert_equal(creep, {'audio_features': [{'danceability': 0.515,
                                         'energy': 0.43,
                                         'key': 7,
                                         'loudness': -9.935,
                                         'mode': 1,
                                         'speechiness': 0.0369,
                                         'acousticness': 0.0102,
                                         'instrumentalness': 0.000141,
                                         'liveness': 0.129,
                                         'valence': 0.104,
                                         'tempo': 91.841,
                                         'type': 'audio_features',
                                         'id': '6b2oQwSGFkzsMtQruIWm2p',
                                         'uri': 'spotify:track:6b2oQwSGFkzsMtQruIWm2p',
                                         'track_href': 'https://api.spotify.com/v1/tracks/6b2oQwSGFkzsMtQruIWm2p',
                                         'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6b2oQwSGFkzsMtQruIWm2p',
                                         'duration_ms': 238640,
                                         'time_signature': 4}]})

**Retrieving playlist information**

In [8]:
playlist_id = '3viCfvZDw1b48dciv60bUq'

In [9]:
endpoint = 'playlists/' + playlist_id
url = base_url + endpoint

In [10]:
playlist = rq.get(url, headers = headers).json()

Retrieving general track info

In [11]:
album, artists, disc_number, duration_ms, explicit, name, popularity, release_date, track_number, uri, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, followers, genres, artist_popularity = [], [], [], [], [], [], [], [],[], [], [], [],[], [], [], [],[], [], [], [], [], [], [], []
artist_ids, audio_ids = '', ''

for item in playlist['tracks']['items']:
    track_info = item['track']
    
    album.append(track_info['album']['name'])
    artists.append(track_info['artists'][0]['name'])
    disc_number.append(track_info['disc_number'])
    duration_ms.append(track_info['duration_ms'])
    explicit.append(track_info['explicit'])
    name.append(track_info['name'])
    popularity.append(track_info['popularity'])
    track_number.append(track_info['track_number'])
    uri.append(track_info['uri'])
    
    artist_ids += track_info['artists'][0]['id'] + ','
    audio_ids += track_info['id'] + ","

Retrieving audio features info

In [12]:
audio_ids = audio_ids.rstrip(',')
audio_url = base_url + "audio-features/?ids=" + audio_ids
response_audio = rq.get(audio_url, headers = headers).json()

for audio_info in response_audio['audio_features']:
    danceability.append(audio_info['danceability'])
    energy.append(audio_info['energy'])
    key.append(audio_info['key'])
    loudness.append(audio_info['loudness'])
    mode.append(audio_info['mode'])
    speechiness.append(audio_info['speechiness'])
    acousticness.append(audio_info['acousticness'])
    instrumentalness.append(audio_info['instrumentalness'])
    liveness.append(audio_info['liveness'])
    valence.append(audio_info['valence'])
    tempo.append(audio_info['tempo'])

Retrieving artist info

In [13]:
artist_ids = artist_ids.rstrip(',')

#split artist_ids string into two halves because of 50 ID cap per request
ids1, ids2 = artist_ids[:int(len(artist_ids)/2)], artist_ids[int(len(artist_ids)/2):]
ids1 = ids1.rstrip(',')
ids2 = ids2.lstrip(',')

artist_url = base_url + 'artists/?ids=' + ids1
response_artists = rq.get(artist_url, headers = headers).json()

for artist_info in response_artists['artists']:
    followers.append(artist_info['followers']['total'])
    genres.append(artist_info['genres'][0])
    artist_popularity.append(artist_info['popularity'])
    
artist_url = base_url + 'artists/?ids=' + ids2
response_artists = rq.get(artist_url, headers = headers).json()

for artist_info in response_artists['artists']:
    followers.append(artist_info['followers']['total'])
    genres.append(artist_info['genres'][0])
    artist_popularity.append(artist_info['popularity'])

Storing as pandas dataframe

In [14]:
data = {'album': album, 'artists': artists, 'disc_number': disc_number, 'duration_ms': duration_ms, 'explicit': explicit, 'name': name, 'popularity': popularity, 'track_number': track_number, 'uri': uri, 'danceability': danceability, 'energy': energy, 'key': key, 'loudness': loudness, 'mode': mode, 'speechiness': speechiness, 'acousticness': acousticness, 'instrumentalness': instrumentalness, 'liveness': liveness, 'valence': valence, 'tempo': tempo, 'followers': followers, 'genres': genres, 'artist_popularity': artist_popularity}
df = pd.DataFrame(data)
df = df.reindex(sorted(df.columns), axis=1)
df.head()

,acousticness,album,artist_popularity,artists,danceability,disc_number,duration_ms,energy,explicit,followers,genres,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,track_number,uri,valence
0,0.752,Paris Milonga,57,Paolo Conte,0.625,1,166160,0.472,False,150203,classic italian pop,0.00298,5,0.1540,-16.407,1,Via con me,62,0.1530,163.410,5,spotify:track:5WHWPRAkR2WYH1CjKV9Omh,0.739
1,0.700,My Way (Expanded Edition),85,Frank Sinatra,0.401,1,276800,0.381,False,4351617,adult standards,0.00000,2,0.1220,-7.458,1,My Way,74,0.0274,75.454,6,spotify:track:3spdoTYpuCpmq19tuD0bOe,0.229
2,0.845,Come Fly With Me (Remastered),85,Frank Sinatra,0.574,1,199093,0.338,False,4351617,adult standards,0.00000,6,0.1650,-11.376,1,Come Fly With Me - Remastered,70,0.0420,67.008,1,spotify:track:4hHbeIIKO5Y5uLyIEbY9Gn,0.493
3,0.453,Sinatra/Basie: The Complete Reprise Studio Rec...,85,Frank Sinatra,0.668,1,147000,0.260,False,4351617,adult standards,0.00000,7,0.0621,-14.256,1,Fly Me To The Moon (In Other Words),75,0.0523,119.416,11,spotify:track:5b7OgznPJJr1vHNYGyvxau,0.368
4,0.419,Les Lacs Du Connemara,58,Michel Sardou,0.668,1,255226,0.786,False,317726,chanson,0.00000,5,0.5210,-10.272,1,Être une femme,54,0.0840,129.359,7,spotify:track:3cXteS7JTe3VtA8OIoWwAb,0.746


Exporting to csv file

In [15]:
df.to_csv('spotify.csv', sep=',')